In [ ]:
# -- necessary setup

import os,sys
sys.path.append(os.path.abspath('../src'))

from IO.RNA_IO import RNA_IO
from utils import parse_pdb_files
from Transformations.transformers.SecondaryStructure import SecondaryStructure

In [19]:
X, Y= parse_pdb_files(['7eaf'])

In [20]:
ss1 = SecondaryStructure()
X1, Y1= ss1.transform(X, Y)
print(X1.shape)
print(Y1["Original"].shape)
print("".join(Y1["SecondaryStructure"][0]))

(1, 94)
(1, 94, 2, 3)
.(((.((..((.(.(((((.(.(((.(()()(.)(.)).(()((...(().(.)..(()).()(....(()(()...)..(.).).)(.)()).


In [21]:
ss2 = SecondaryStructure(nussinov=True)
X2, Y2= ss2.transform(X1, Y1)
print(X2.shape)
print(Y2["Original"].shape)
print("".join(Y2["SecondaryStructure"][0]))


(1, 94)
(1, 94, 2, 3)
..(((()(..)).))..().(((.)((())())((.)(((())(..((.)..()))(()(.((((((())()))))(()))))..)()).))).


In [22]:
from viennarna import RNA
import matplotlib.pyplot as plt

#Visualize the structure with RNAplot
fig, ax = plt.subplots()
ax.set_title(f"RNA Secondary Structure: {X}")
RNA.plot(X, X1, ax=ax)
plt.show()

ModuleNotFoundError: No module named 'viennarna'

In [ ]:
fig, ax = plt.subplots()
ax.set_title(f"RNA Secondary Structure: {X}")
RNA.plot(X, X2, ax=ax)
plt.show()